In [2]:
import datetime
import geopandas
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import tensorflow as tf

2023-04-21 08:09:27.963791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# TODO try only with best performing numeric columns for input into model first
# TODO select features for deep learning model after ranking features with random forest (feature importance)
# TODO lstm network
# TODO SMOTE algorithm, for upsampling havested fields; we only have ~300 :(

In [3]:
#local scripts
from scripts import veg_indices, utilities, plots
#from scripts.utilities import *

In [4]:
# cpied from learning_about-data.ipynb
BANDS_DICT = {   'B2': 'Blue',
            'B3': 'Green',
            'B4': 'Red',
            'B5': 'Red_Edge_1',
            'B6': 'Red_Edge_2',
            'B7': 'Red_Edge_3',
            'B8': 'NIR',
            'B8A': 'Red_Edge_4',
            'B11': 'SWIR_1',
            'B12': 'SWIR_2'}

BANDS = list(BANDS_DICT.keys())

In [5]:
# cpied from learning_about-data.ipynb
DF = geopandas.read_file('../data/merged_images.geojson')
DF.rename(columns = {'is_within_period':'har_evnt'}, inplace = True)
NUM_SAMPLES = len(np.unique(DF.image_idx)) - 1

In [6]:
# cpied from learning_about-data.ipynb
df = DF.copy()
df = df[(df.NDVI) != 0] # drop invalid points
VEG_INDICES_NAMES = veg_indices.add_veg_indices(df) + ['NDVI'] 
VEG_DIFF_NAMES = veg_indices.add_veg_diff(df, VEG_INDICES_NAMES)
NUMERIC_COLS = BANDS + VEG_INDICES_NAMES + VEG_DIFF_NAMES
df = df.dropna(subset=['sample_idx']) # removes the first image, because sample_idx is NaN there
df.columns, df.shape

Added:  ['DVI', 'RVI', 'ARVI', 'NDI45', 'GNDVI', 'MCARI', 'MTCI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI']
Added:  ['sample_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'MTCI_diff', 'NDTI_diff', 'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff', 'NDWI_diff', 'NDSI_diff', 'NDVI_diff']


(Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
        'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
        'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'NDI45',
        'GNDVI', 'MCARI', 'MTCI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI',
        'NDWI', 'NDSI', 'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff',
        'ARVI_diff', 'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'MTCI_diff',
        'NDTI_diff', 'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff',
        'BSI_diff', 'NDWI_diff', 'NDSI_diff', 'NDVI_diff', 'sample_idx'],
       dtype='object'),
 (6678, 54))

In [11]:
# TODO look at samples(aka image2s) one by one

# each sample_idx is an sample
samples = np.array([1, 2, 3])




In [7]:
np.concatenate([sample])

NameError: name 'sample' is not defined

In [12]:
samples[..., np.newaxis]

array([[1],
       [2],
       [3]])

In [8]:
np.newaxis

In [13]:
samples

array([1, 2, 3])

In [24]:
np.reshape(samples, samples.shape + (1,))

array([[1],
       [2],
       [3]])

In [51]:
sample_0 = np.array(df[df["sample_idx"] == 's0'][["B2"]].head())

In [52]:
zero = np.reshape(sample_0, sample_0.shape + (1,))

In [53]:
sample_1 = np.array(df[df["sample_idx"] == 's1'][["B2"]].head())
one = np.reshape(sample_1, sample_1.shape + (1,))

In [56]:
np.concatenate([zero, one])

array([[[0.8832 ]],

       [[0.0392 ]],

       [[0.0294 ]],

       [[0.86005]],

       [[0.0455 ]],

       [[0.0282 ]],

       [[0.0226 ]],

       [[0.025  ]],

       [[0.0338 ]],

       [[0.0425 ]]])

In [20]:
lookupTable, indexed_dataSet = np.unique(df.har_evnt, return_inverse=True)
np.sum(tf.keras.utils.to_categorical(indexed_dataSet), axis=0)

array([6350.,  328.], dtype=float32)

In [35]:
tpls = tf.keras.utils.to_categorical(indexed_dataSet)
np.reshape(tpls, ( (2, -1)))

array([[1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [42]:
np.sum(np.array(list(zip(*tpls))), axis=1)


array([6350.,  328.], dtype=float32)

In [31]:
tpls.shape

(6678, 2)

In [16]:
df[df['point_idx'] == "p10"][["sample_idx", "point_idx", "har_evnt", "start_date", "end_date", "finHarvDat"]]

,sample_idx,point_idx,har_evnt,start_date,end_date,finHarvDat
568,s0,p10,False,2022-01-29,2022-02-18,2022-07-14
1126,s1,p10,False,2022-02-19,2022-03-11,2022-07-14
1684,s2,p10,False,2022-03-12,2022-04-01,2022-07-14
2242,s3,p10,False,2022-04-02,2022-04-22,2022-07-14
2800,s4,p10,False,2022-04-23,2022-05-13,2022-07-14
3358,s5,p10,False,2022-05-14,2022-06-03,2022-07-14
3916,s6,p10,False,2022-06-04,2022-06-24,2022-07-14
4474,s7,p10,True,2022-06-25,2022-07-15,2022-07-14
5032,s8,p10,False,2022-07-16,2022-08-05,2022-07-14
5590,s9,p10,False,2022-08-06,2022-08-26,2022-07-14


In [43]:
np.sum(df.har_event)

AttributeError: 'GeoDataFrame' object has no attribute 'har_event'